In [1]:
import numpy as np
import time

# Tutorial: Higher Order Array Manipulation

For the sake of simplicity, we define $n$-th order tensors as arrays of dimension $n$. A $0$-th order array is a scalar, a $1$-st order array is a vector in $\mathbb{R}^{d_1}$, and a $2$-nd order array is a matrix in $\mathbb{R}^{d_1\times d_2}$. Going further, a $n$-th order array is an element of $\mathbb{R}^{d_1\times...\times d_n}$ for some dimensions $(d_i)_{i\in[n]}$.

## Declaration

In [2]:
# Declare a third order array 
d1, d2, d3 = 2, 3, 5
A = np.random.rand(d1, d2, d3)

print("The shape of A is {}".format(A.shape))

The shape of A is (2, 3, 5)


## Indexing

Say we have a $3$-rd order array $\mathbf{A}\in\mathbb{R}^{d_1\times d_2\times d_3}$. Indexing and slicing works as for lower order arrays:

In [3]:
print("A[0] has shape {}".format(A[0].shape))
print("A[:, 1:, :] has shape {}".format(A[:, 1:, :].shape))
print("A[:, 1, 2:4] has shape {}".format(A[:, 1, 2:4].shape))

A[0] has shape (3, 5)
A[:, 1:, :] has shape (2, 2, 5)
A[:, 1, 2:4] has shape (2, 2)


We can also use a different indexing array $\mathbf{b}$ to index $\mathbf{A}$. This indexing operates on the first dimension of $\mathbf{A}$, meaning that if $\mathbf{b}\in\mathbb{R}^{l_1\times l_2}$, then `A[b]` will have shape $l_1\times l_2\times d_2\times d_3$.

In [4]:
b = np.array([0, 0, 1, 0])
print("If A has shape {}, b has shape {}, then A[b] has shape {}.".format(A.shape, b.shape, A[b].shape))

b = np.array([[0, 0, 1, 0], [1, 1, 0, 1]])
print("If A has shape {}, b has shape {}, then A[b] has shape {}.".format(A.shape, b.shape, A[b].shape))

If A has shape (2, 3, 5), b has shape (4,), then A[b] has shape (4, 3, 5).
If A has shape (2, 3, 5), b has shape (2, 4), then A[b] has shape (2, 4, 3, 5).


... This works provided the indexing array $\mathbf{b}$ has integer values comprised between $0$ and $d_{1}-1$ (included).

In [5]:
try:
    b = np.array([0, 0, 2, 0])
    A[b]
except Exception as e:
    print("We have an out-of bound indexing: d_1=1 but max b=2")
    print("The exception is: {}".format(e))

We have an out-of bound indexing: d_1=1 but max b=2
The exception is: index 2 is out of bounds for axis 0 with size 2


## Operations

Imagine now that we have a batch of $1000$ $d\times d$ matrices: $(\mathbf{a}_i)_{i\in[1000]}$, for which we want to compute the trace. We could loop over the matrices and compute the traces separately.

In [6]:
d = 2

ais    = [np.random.rand(d, d) for i in range(1000)]

start  = time.time()
traces = [np.trace(ai) for ai in ais]
end    = time.time()

print("Elapsed time: {:.2e}s.".format(end - start))

Elapsed time: 1.17e-02s.


Alternatively, we could vectorize this operation using a three dimensional array $\mathbf{A}\in\mathbb{R}^{1000\times d\times d}$ that contains the stacked matrices.

In [7]:
A = np.stack(ais, axis=0)

print("A has shape {}".format(A.shape))

start  = time.time()
traces = np.trace(A, axis1=1, axis2=2)
end    = time.time()

print("Elapsed time: {:.2e}s.".format(end - start))

A has shape (1000, 2, 2)
Elapsed time: 6.13e-04s.


And we reduced the computation time by an order of magnitude! A different option that we will use extensively during part 3 is to use [Einstein summation](https://en.wikipedia.org/wiki/Einstein_notation). For the traces computation this would be written: $\mathbf{A}_{i,j,j}$. This can be done with Numpy with the method [`np.einsum`](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html).

In [8]:
start  = time.time()
traces = np.einsum('ijj->i', A)
end    = time.time()

print("Elapsed time: {:.2e}s.".format(end - start))

Elapsed time: 1.79e-03s.


As efficient as the trace method! Also, `np.einsum` is highly flexible. It can compute the transpose of a batch of arrays, or various kinds of matrix multiplications.

In [9]:
ais = [np.random.rand(2, 3) for i in range(1000)]

# Transpose each stacked matrices
A = np.stack(ais, axis=0)
print("A has shape {}".format(A.shape))
AT = np.einsum('ijk -> ikj', A)
print("A^T has shape {}".format(AT.shape))

A has shape (1000, 2, 3)
A^T has shape (1000, 3, 2)


Below we show how to compute $\mathbf{a}_i^T\mathbf{a}_i$ for some matrices $\mathbf{a}_i\in\mathbb{R}^{2\times 3}$ using `np.einsum`.

In [10]:
print("A has shape {}".format(A.shape))
product_As = np.einsum('ijk, ijn -> ikn', A, A)
print("Stacked ai^T.ai has shape {}".format(product_As.shape))

A has shape (1000, 2, 3)
Stacked ai^T.ai has shape (1000, 3, 3)


To scale each $\mathbf{a}_i$ by a weight $w_i$, we can still use `np.einsum`. Define the vector containing all the weights $\mathbf{w}=(w_i)_i$, we have:

In [11]:
w = np.random.rand(A.shape[0])
weighted_A = np.einsum('i, ijk -> ijk', w, A)
print("Weighted and stacked ai has shape {}".format(weighted_A.shape))

Weighted and stacked ai has shape (1000, 2, 3)


In [136]:
import numpy as np

a = np.array([[[1, 2, 3], 
               [4, 5, 6], 
               [7, 8, 9]],
              [[1, 2, 4],
               [4, 5, 6],
               [7, 8, 9]]])


# matrix multiplication
np.einsum('ijk, ikn -> ijn', a, a)

array([[[ 30,  36,  42],
        [ 66,  81,  96],
        [102, 126, 150]],

       [[ 37,  44,  52],
        [ 66,  81, 100],
        [102, 126, 157]]])

In [132]:
c = np.array([[1, 2, 4], 
              [4, 5, 6], 
              [7, 8, 9]])

c @ c

array([[ 37,  44,  52],
       [ 66,  81, 100],
       [102, 126, 157]])

In [134]:
# transposed first matrix multiplication
np.einsum('ijk, ijn -> ikn', a, a)

array([[[ 66,  78,  90],
        [ 78,  93, 108],
        [ 90, 108, 126]],

       [[ 66,  78,  91],
        [ 78,  93, 110],
        [ 91, 110, 133]]])

In [135]:
c.T @ c

array([[ 66,  78,  91],
       [ 78,  93, 110],
       [ 91, 110, 133]])